In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.pauli import group_observables

from scipy.optimize import minimize

# General imports
import os
import json
import numpy as np
from datetime import datetime, timedelta

# custom module
from susy_qm import calculate_Hamiltonian


from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer.noise import NoiseModel

In [27]:
service = QiskitRuntimeService(name="NQCC-Q3")
real_backend = service.backend('ibm_kingston')
noise_model = NoiseModel.from_backend(real_backend)

In [31]:
potential = 'QHO'
cutoff = 2
shots = None

In [32]:
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]
min_eigenvalue = min(eigenvalues.real)

num_qubits = int(1 + np.log2(cutoff))

H_decomp = qml.pauli_decompose(H, wire_order=range(num_qubits))
paulis = H_decomp.ops
coeffs = H_decomp.coeffs

groups = group_observables(paulis)

In [33]:
def cost_function(params):

    dev = qml.device('qiskit.aer', wires=num_qubits, noise_model=noise_model, shots=shots, seed=42)
  
    @qml.qnode(dev)
    def circuit(params, groups):
    
        ############### QHO ##########################
        #basis = [1] + [0]*(num_qubits-1)
        #qml.BasisState(basis, wires=range(num_qubits))
        qml.X(0)
        qml.RY(params[0], wires=[0])      
       
        return [qml.expval(op) for op in groups]
    
    energy = 0
    for group in groups:
        results = circuit(params, group)
        for op, res in zip(group, results):
            idx = paulis.index(op)
            energy += coeffs[idx] * res

    return energy
    

In [34]:
seed=42
num_params = 1

max_iter = 10000
initial_tr_radius = 0.8
final_tr_radius = 1e-11

np.random.seed(seed)
x0 = np.random.random(size=num_params)*2*np.pi
bounds = [(0, 2 * np.pi) for _ in range(num_params)]


res = minimize(
        cost_function,
        x0,
        bounds=bounds,
        method= "COBYQA",
        options= {
            'maxiter':max_iter, 
            'maxfev':max_iter, 
            'initial_tr_radius':initial_tr_radius, 
            'final_tr_radius':final_tr_radius, 
            'scale':True, 
            'disp':True}
    )

Starting the optimization procedure.
Initial trust-region radius: 0.8.
Final trust-region radius: 1e-11.
Maximum number of function evaluations: 10000.
Maximum number of iterations: 10000.



c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\pennylane_qiskit\qiskit_device_legacy.py:132: UserWarning: The analytic calculation of expectations, variances and probabilities is only supported on statevector backends, not on the aer_simulator. Such statistics obtained from this device are estimates based on samples.
  warnings.warn(self.analytic_warning_message.format(backend), UserWarning)


cost_function([ 2.513e+00]) = 0.8818359375
cost_function([ 5.027e+00]) = 0.357421875
cost_function([ 0.000e+00]) = 0.044921875

New trust-region radius: 0.08000000000000002.
Number of function evaluations: 3.
Number of iterations: 3.
Least value of cost_function: 0.044921875.
Maximum constraint violation: 0.0.
Corresponding point: [ 0.000e+00].

cost_function([ 2.513e-01]) = 0.064453125
cost_function([ 1.257e-01]) = 0.0439453125

New trust-region radius: 0.008000000000000002.
Number of function evaluations: 5.
Number of iterations: 9.
Least value of cost_function: 0.0439453125.
Maximum constraint violation: 0.0.
Corresponding point: [ 1.257e-01].

cost_function([ 6.854e-02]) = 0.0390625
cost_function([ 0.000e+00]) = 0.041015625
cost_function([ 1.142e-02]) = 0.0390625
cost_function([ 3.998e-02]) = 0.0419921875
cost_function([ 9.368e-02]) = 0.0400390625

New trust-region radius: 0.0008000000000000003.
Number of function evaluations: 10.
Number of iterations: 17.
Least value of cost_funct

In [31]:
print(min_eigenvalue, energies)

0.03201011000923872 [np.float64(39.86011041287141)]


In [19]:
# Save run
run = {
    #"starttime": starttime,
    "potential": potential,
    "cutoff": cutoff,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "num_VQE": num_vqe_runs,
    "shots": shots,
    "Optimizer": {
        "name": "differential_evolution",
        "bounds": "[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]",
        "maxiter": max_iter,
        "tolerance": tol,
        "abs_tolerance": abs_tol,
        "strategy": strategy,
        "popsize": popsize,
        'init': 'scaled_samples',
    },
    "results": energies,
    "params": [x.tolist() for x in x_values],
    "num_iters": num_iters,
    "num_evaluations": num_evaluations,
    "success": np.array(success, dtype=bool).tolist(),
    "run_times": [str(x) for x in run_times],
    "parallel_run_time": str(vqe_time),
    "total_VQE_time": str(total_run_time)
}

run

{'potential': 'AHO',
 'cutoff': 16,
 'exact_eigenvalues': [-0.001166975680574785,
  1.6774941961816563,
  1.6863812488131036,
  4.364321396210466],
 'ansatz': 'StronglyEntanglingLayers-1layer',
 'num_VQE': 1,
 'shots': 2,
 'Optimizer': {'name': 'differential_evolution',
  'bounds': '[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
  'maxiter': 100,
  'tolerance': 0.001,
  'abs_tolerance': 0.001,
  'strategy': 'randtobest1bin',
  'popsize': 5,
  'init': 'scaled_samples'},
 'results': [np.float64(-0.0011669756806017494)],
 'params': [[1.3439955151618823, 1.6870449623891193, 1.55988406008951]],
 'num_iters': [100],
 'num_evaluations': [605],
 'success': [False],
 'run_times': ['0:07:23.761694'],
 'parallel_run_time': '0:07:23.762647',
 'total_VQE_time': '0:07:23.761694'}

In [22]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\Noise Simulator\NoiseSimFiles\\{}\\".format(potential)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)